In [ ]:
import pandas as pd

# Load the datasets
file_1990_1999 = pd.read_csv('D:/GUVI_Projects/My_Projects/singapur sheets/ResaleFlatPricesBasedonApprovalDate19901999.csv')
file_2000_2012 = pd.read_csv('D:/GUVI_Projects/My_Projects/singapur sheets/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv')
file_2012_2014 = pd.read_csv('D:/GUVI_Projects/My_Projects/singapur sheets/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv')
file_2015_2016 = pd.read_csv('D:/GUVI_Projects/My_Projects/singapur sheets/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv')
file_2017_onwards = pd.read_csv('D:/GUVI_Projects/My_Projects/singapur sheets/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

# Standardize column names across datasets
file_1990_1999.columns = file_1990_1999.columns.str.lower().str.replace(' ', '_')
file_2000_2012.columns = file_2000_2012.columns.str.lower().str.replace(' ', '_')
file_2012_2014.columns = file_2012_2014.columns.str.lower().str.replace(' ', '_')
file_2015_2016.columns = file_2015_2016.columns.str.lower().str.replace(' ', '_')
file_2017_onwards.columns = file_2017_onwards.columns.str.lower().str.replace(' ', '_')

# Find common columns across all datasets
common_columns = list(set(file_1990_1999.columns) & set(file_2000_2012.columns) & set(file_2012_2014.columns) & set(file_2015_2016.columns) & set(file_2017_onwards.columns))

# Select only common columns in each DataFrame
file_1990_1999 = file_1990_1999[common_columns]
file_2000_2012 = file_2000_2012[common_columns]
file_2012_2014 = file_2012_2014[common_columns]
file_2015_2016 = file_2015_2016[common_columns]
file_2017_onwards = file_2017_onwards[common_columns]

# Concatenate all datasets
all_files = pd.concat([file_1990_1999, file_2000_2012, file_2012_2014, file_2015_2016, file_2017_onwards], ignore_index=True)

# Display the combined dataset
print(all_files.head())

# Save the combined dataset to a CSV file
all_files.to_csv('D:/Final_Projects/project_new/Project1_resale_flat_prices.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import logging

# Setup logging
logging.basicConfig(filename="error_log.log", level=logging.DEBUG,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(use_label_encoder=False, eval_metric='rmse')
}

# Step 1: Load the Dataset
dataset_path = 'D:/Final_Projects/project_new/Project1_resale_flat_prices.csv'
data = pd.read_csv(dataset_path)

# Step 2: Initial Exploration of the Dataset
print("Initial Dataset:")
print(data.head())

# Step 3: Inspect the Dataset
print("\nDataset Info:")
data.info()
print("\nDescriptive Statistics:")
print(data.describe())
print("\nColumn Data Types:")
print(data.dtypes)

# Step 4: Check for Missing Values
print("\nMissing Values in Each Column:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])  # Only show columns with missing values

# Step 5: Handle Missing Values
data = data.dropna()  # Consider using data.fillna() for more control

# Display the dataset info again to ensure there are no missing values
print("\nDataset Info After Handling Missing Values:")
data.info()

# Step 6: Data Type Conversions
data['month'] = pd.to_datetime(data['month'])

# Step 7: Feature Extraction and Engineering
data['year'] = data['month'].dt.year
data['month'] = data['month'].dt.month

# Display the first few rows to verify the new features
print("\nDataset After Feature Engineering:")
print(data.head())

# Step 8: Standardize unique values in 'flat_model' and 'flat_type' columns
# Define mappings to handle variants (e.g., "multi-generation" vs "multi generation")
flat_model_map = {
    'multi-generation': 'multi_generation', 
    'multi generation': 'multi_generation'
}

flat_type_map = {
    '5 room': '5_room',
    '5-room': '5_room',
    '3 room': '3_room',
    '3-room': '3_room',
    'executive apartment': 'executive_apartment',
    'executive maisonette': 'executive_maisonette',
}

# Apply standardization
data['flat_model'] = data['flat_model'].str.lower().replace(flat_model_map)
data['flat_type'] = data['flat_type'].str.lower().replace(flat_type_map)

# Step 9: Verify the unique values after standardization
print("\nUnique flat_model values after standardization:", data['flat_model'].unique())
print("Unique flat_type values after standardization:", data['flat_type'].unique())

# Step 10: Exploratory Data Analysis (EDA)
# Step A: Visualize distributions of numerical features
plt.figure(figsize=(12, 6))
sns.histplot(data['resale_price'], kde=True, bins=50)
plt.title('Distribution of Resale Prices')
plt.xlabel('Resale Price')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(12, 6))
sns.histplot(data['floor_area_sqm'], kde=True, bins=50)
plt.title('Distribution of Floor Area (sqm)')
plt.xlabel('Floor Area (sqm)')
plt.ylabel('Frequency')
plt.show()

# Step B: Visualize correlations
plt.figure(figsize=(12, 8))
numeric_data = data.select_dtypes(include=[np.number])
correlation_matrix = numeric_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

# Step C: Visualize relationships between features and target
plt.figure(figsize=(12, 6))
sns.scatterplot(x='floor_area_sqm', y='resale_price', data=data)
plt.title('Resale Price vs. Floor Area (sqm)')
plt.xlabel('Floor Area (sqm)')
plt.ylabel('Resale Price')
plt.show()

# Step 11: Save the Cleaned Data
cleaned_dataset_path = 'D:/Final_Projects/project_new/latest_cleaned_singapore_resale_flat_prices.csv'
data.to_csv(cleaned_dataset_path, index=False)
print(f"\nCleaned dataset saved at: {cleaned_dataset_path}")

# Step 12: Prepare the Features and Target Variable
data = pd.read_csv(cleaned_dataset_path)

X = data[['floor_area_sqm', 'storey_range', 'lease_commence_date', 'year', 'month', 'flat_model', 'town', 'flat_type', 'street_name']]
y = data['resale_price']

# Step 13: Encode Categorical Variables
X = pd.get_dummies(X, columns=['storey_range', 'flat_model', 'town', 'flat_type', 'street_name'], drop_first=True)

# Display the first few rows of the encoded features
print("\nEncoded Features (X):")
print(X.head())

# Step 14: Split the Dataset into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print("\nShapes of Training and Testing Sets:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

# Step 15: Define and Train Machine Learning Models
results = {}  # Initialize results dictionary

for model_name, model in models.items():
    try:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate the model
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mse ** 0.5  # Calculate RMSE from MSE
        r2 = r2_score(y_test, y_pred)

        logging.debug(f"{model_name} - MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")
        print(f"\n{model_name} - MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

        # Store results
        results[model_name] = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2,
        }

    except Exception as e:
        logging.error(f"Error training {model_name}: {str(e)}")

# Step 16: Cross-Validation for the Best Model

if results:
    best_model_name = max(results, key=lambda x: results[x]['R2'])  # Choose the model with the highest R2

    try:
        best_model = models[best_model_name]

        # Perform k-fold cross-validation on the best model
        kfold = KFold(n_splits=5, random_state=42, shuffle=True)
        cv_scores = []

        for train_index, val_index in kfold.split(X):
            X_train_cv, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train_cv, y_val = y.iloc[train_index], y.iloc[val_index]

            best_model.fit(X_train_cv, y_train_cv)
            y_pred_cv = best_model.predict(X_val)
            score = r2_score(y_val, y_pred_cv)
            cv_scores.append(score)

        logging.debug(f"Cross-Validation R2 Scores for {best_model_name}: {cv_scores}")
        logging.debug(f"Mean Cross-Validation R2 Score: {np.mean(cv_scores):.2f}")
        
        print(f"\nCross-Validation R2 Scores for {best_model_name}: {cv_scores}")
        print(f"Mean Cross-Validation R2 Score: {np.mean(cv_scores):.2f}")

    except Exception as e:
        logging.error(f"Error performing cross-validation for {best_model_name}: {str(e)}")
else:
    print("No results available to determine the best model.")

# Step 17: Save the Best Model

if 'best_model_name' in locals():  # Check if best_model_name is defined
    model_path = f'D:/Final_Projects/project_new/{best_model_name.replace(" ", "_").lower()}_model.joblib'

    try:
        joblib.dump(best_model, model_path)
        logging.debug(f"Best Model saved at: {model_path}")
        print(f"\nBest Model saved at: {model_path}")

    except Exception as e:
        logging.error(f"Error saving best model: {str(e)}")
else:
    print("No best model to save.")


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import logging

# Setup logging
logging.basicConfig(filename="error_log.log", level=logging.DEBUG,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Step 1: Load the Dataset
dataset_path = 'D:/Final_Projects/project_new/Project1_resale_flat_prices.csv'
data = pd.read_csv(dataset_path, low_memory=False)  # Optimize memory usage by setting low_memory

# Step 2: Remove Outliers in Resale Price (using IQR method)
Q1 = data['resale_price'].quantile(0.25)
Q3 = data['resale_price'].quantile(0.75)
IQR = Q3 - Q1
outlier_condition = ~((data['resale_price'] < (Q1 - 1.5 * IQR)) | 
                      (data['resale_price'] > (Q3 + 1.5 * IQR)))
data = data.loc[outlier_condition]

# Step 3: Handle Missing Values (drop them for simplicity in this case)
data.dropna(inplace=True)

# Step 4: Feature Engineering
data['month'] = pd.to_datetime(data['month'])
data['year'] = data['month'].dt.year
data['month'] = data['month'].dt.month

# Step 5: Standardize Categorical Variables (in-place for space efficiency)
flat_model_map = {
    'multi-generation': 'multi_generation',
    'multi generation': 'multi_generation'
}

flat_type_map = {
    '5 room': '5_room',
    '5-room': '5_room',
    '3 room': '3_room',
    '3-room': '3_room',
}

data['flat_model'] = data['flat_model'].str.lower()
data['flat_model'].replace(flat_model_map, inplace=True)

data['flat_type'] = data['flat_type'].str.lower()
data['flat_type'].replace(flat_type_map, inplace=True)

# Step 6: Verify the unique values after standardization
print("\nUnique flat_model values after standardization:", data['flat_model'].unique())
print("Unique flat_type values after standardization:", data['flat_type'].unique())

# Step 7: Save the Cleaned Data
cleaned_dataset_path = 'D:/Final_Projects/project_new/latest_cleaned_singapore_resale_flat_prices.csv'
data.to_csv(cleaned_dataset_path, index=False)
print(f"\nCleaned dataset saved at: {cleaned_dataset_path}")

# Step 8: Prepare the Features and Target Variable
X = data[['floor_area_sqm', 'storey_range', 'lease_commence_date', 'year', 'month', 'flat_model', 'town', 'flat_type']]
y = data['resale_price']

# Step 9: Encode Categorical Variables using One-Hot Encoding (minimizing memory usage)
X = pd.get_dummies(X, columns=['storey_range', 'flat_model', 'town', 'flat_type'], drop_first=True, dtype=np.uint8)

# Step 10: Split the Dataset into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 11: Define the RandomForestRegressor Model
model = RandomForestRegressor(random_state=42, n_jobs=-1)

# Step 12: Define Hyperparameter Grid (reduce size to optimize time)
param_grid = {
    'n_estimators': [100, 200],  # Reduce the number of estimators
    'max_depth': [10, 20],       # Focus on a smaller range of depths
    'min_samples_split': [2, 5],  # Use fewer splits to reduce search time
    'min_samples_leaf': [1, 2]
}

# Step 13: Use GridSearchCV for Hyperparameter Tuning (Optimized for Time and Memory)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=3,  # Reduce cross-validation folds to save time
    n_jobs=-1,  # Utilize all cores
    verbose=1
)

# Fit the grid search model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Step 14: Make Predictions on the Test Set
y_pred = best_model.predict(X_test)

# Step 15: Evaluate the Model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

logging.debug(f"Random Forest - MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")
print(f"\nRandom Forest - MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

# Step 16: Save the Best Model
model_path = 'D:/Final_Projects/project_new/random_forest_best_model.joblib'
joblib.dump(best_model, model_path, compress=3)  # Compress the model to save space
logging.debug(f"Best Model saved at: {model_path}")
print(f"\nBest Model saved at: {model_path}")



Unique flat_model values after standardization: ['improved' 'new generation' 'model a' 'standard' 'simplified'
 'model a-maisonette' 'apartment' 'maisonette' 'terrace' '2-room'
 'improved-maisonette' 'multi_generation' 'premium apartment'
 'adjoined flat' 'premium maisonette' 'model a2' 'dbss' 'type s1'
 'premium apartment loft' '3gen']
Unique flat_type values after standardization: ['1 room' '3_room' '4 room' '5_room' '2 room' 'executive'
 'multi generation' 'multi-generation']

Cleaned dataset saved at: D:/Final_Projects/project_new/latest_cleaned_singapore_resale_flat_prices.csv
Fitting 3 folds for each of 16 candidates, totalling 48 fits

Random Forest - MAE: 20096.53, MSE: 862841410.73, RMSE: 29374.16, R2: 0.96

Best Model saved at: D:/Final_Projects/project_new/random_forest_best_model.joblib
